# Interactive Brokers Connection Example

This notebook demonstrates basic connection and operations with Interactive Brokers using the ib_async library.

## Connection

**Important Note**: From now on, use `ib_async` instead of the older Interactive Brokers API.

In [3]:
from ib_async import *
util.startLoop()  # required in Jupyter (interactive environments), not in scripts

In [4]:
ib = IB()

**Make sure you are logged in your TWS!!**

In [5]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [6]:
ib.positions()

[]

In [7]:
ib.disconnect()

'Disconnecting from 127.0.0.1:7497, 117 B sent in 8 messages, 18.9 kB received in 398 messages, session time 10.5 s.'